### LLM OpenAeroStruct

#### A multiagent tool to write optimization code for OpenAeroStruct with simply high level inputs from the user, i.e. I want to design a sweeped rectangular wing with elliptical lift distribution

In [1]:
# Import the generative ai library
import google.generativeai as genai #type: ignore

# Import all the modules necessary to run OpenAeroStruct and paths
import re
import time
import os
import subprocess
import warnings
import numpy as np
import pandas as pd #type: ignore
import openmdao.api as om
import json

# import OpenAeroStruct modules
from openaerostruct.geometry.utils import generate_mesh  # helper functions to generate mesh
from openaerostruct.geometry.geometry_group import Geometry
from openaerostruct.aerodynamics.aero_groups import AeroPoint

# Import the plotting libraries
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import niceplots  # Optional but recommended

#Ignore warnings and use the nice plots style
warnings.filterwarnings("ignore")

plt.style.use(
    niceplots.get_style("james-dark")
)  # Options: "doumont-light", "doumont-dark", "james-light", "james-dark"

In [2]:
#Import the relevant agents
from Agents import ReformulatorAgent
from Agents import BaseMeshAgent
from Agents import GeometryAgent
from Agents import OptimizerAgent
from Agents import ResultsReaderAgent
from Agents import ReportWriter
from Agents import RetrievalAgent
from RAG_Retrieval import run_retriever

In [3]:
# Define the initial sample query from the user.
User_Request = """For this design, we will keep the area at S = 100 m2. The span is b = 10 m. The cruise condition corresponds to CL = 2.0. Your job is to minimize drag at the condition of CL = 2.0 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""

"""HERE ARE SOME BACKUP PROMPTS"""
#User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request  = """For this design, we will keep the area at S = 100 m2. The span is b = 10 m. The cruise condition corresponds to CL = 2.0. Your job is to minimize drag at the condition of CL = 2.0 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper, dihedral, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""

'HERE ARE SOME BACKUP PROMPTS'

In [4]:
error_gen_flag = False
while not error_gen_flag:
    reformulator = ReformulatorAgent()
    reformulator_output = reformulator.execute_task(User_Request)

    if reformulator_output is not None:
        error_gen_flag = True

In [5]:
print("Reformulator Output:")
print(reformulator_output)

Reformulator Output:
{'objective_function': 'Minimize drag', 'trim_condition': 'CL = 2.0', 'geometric_constraint': 'Wing area (S) = 100 m^2, Span (b) = 10 m', 'design_variables': 'Taper, Twist, Sweep', 'baseline_wing_mesh': 'rect', 'optimization_algorithm': 'SLSQP', 'plotting_requirements': 'Elliptical lift distribution', 'errors': ''}


#### Now start writing the file into RunOAS to prepare for the optimization and plotting script using LLM.

In [6]:
#Write the mesh prompt
error_gen_flag = False
while not error_gen_flag:
    MeshPrompt  = f"""For this wing desgin, the geometric parameters are as follows: {reformulator_output["geometric_constraint"]}, and the type of the wing mesh should be: {reformulator_output["baseline_wing_mesh"]}"""
    mesher = BaseMeshAgent()
    mesher_output = mesher.execute_task(MeshPrompt)

    if mesher_output["python_code"] is not None:
        error_gen_flag = True

In [7]:
print(mesher_output["python_code"])
print(mesher_output)

mesh_dict = {
    "num_y": 19, #number of panels in the y direction, 19 is a good starting number
    "num_x": 3, #number of panels in the x direction, 3 is a good starting number
    "wing_type": "rect", #This can either be "rect" or "crm" only
    "symmetry": True, # True if the wing is symmetric, False if it is not, wings are typically symmetric
    "span": 10.0, #This is the full span of the wing in meters
    "root_chord": 10.0, #This is the root chord of the wing in meters
    "span_cos_spacing": 0.0, #This is usually not edited
    "chord_cos_spacing": 0.0, #This is usually not edited
}

# Generate VLM mesh for half-wing
mesh = generate_mesh(mesh_dict)   # this creates a rectangular wing mesh, DO NOT EDIT THIS LINE

# plot mesh
plot_mesh(mesh)  #this plots the rectangular wing mesh, DO NOT EDIT THIS LINE
{'calculations_and_explain': "The task requires generating OpenAeroStruct mesh code for a rectangular wing given the wing area and span. The wing type is specified as 'rect'. I 

In [8]:
#Geometry Setup
error_gen_flag = False
while not error_gen_flag:
    GeometryPrompt  = f"""For this wing desgin, we are allowed to change the following parameters: {reformulator_output["design_variables"]}"""
    geometry_setup = GeometryAgent()
    geometry_output = geometry_setup.execute_task(GeometryPrompt)

    if geometry_output["python_code"] is not None:
        error_gen_flag = True

In [9]:
print(geometry_output["python_code"])
print(geometry_output)

surface = {
    # Wing definition, KEEP THE SAME UNLESS ASKED TO CHANGE
    "name": "wing",  # name of the surface, keep as wing
    "symmetry": True,  # if true, model one half of wing reflected across the plane y = 0
    "S_ref_type": "wetted",  # how we compute the wing area, can be 'wetted' or 'projected'
    "mesh": mesh,

    # Aerodynamic performance of the lifting surface at an angle of attack of 0 (alpha=0).
    # These CL0 and CD0 values are added to the CL and CD obtained from aerodynamic analysis of the surface to get the total CL and CD.
    # These CL0 and CD0 values do not vary wrt alpha. DO NOT EDIT THEM UNLESS ASKED TO.
    "CL0": 0.0,  # CL of the surface at alpha=0
    "CD0": 0.0,  # CD of the surface at alpha=0

    # Airfoil properties for viscous drag calculation, DO NOT CHANGE UNLESS ASKED TO
    "k_lam": 0.05,  # percentage of chord with laminar flow, used for viscous drag
    "c_max_t": 0.303,  # chordwise location of maximum (NACA0015)
    "t_over_c_cp": np.ar

In [10]:
# Optimization Setup
error_gen_flag = False
while not error_gen_flag:
    OptimizerPrompt  = f"""For this wing desgin, the optimization parameters are as follows: {reformulator_output["design_variables"]}, the objective function is {reformulator_output["objective_function"]}, the geometric constraints are {reformulator_output["geometric_constraint"]}, the flight condition is {reformulator_output["trim_condition"]}, and the optimization algorithm is {reformulator_output["optimization_algorithm"]}"""
    optimizer_setup = OptimizerAgent()
    optimizer_output = optimizer_setup.execute_task(OptimizerPrompt)

    if optimizer_output["python_code"] is not None:
        error_gen_flag = True

In [11]:
print(optimizer_output["python_code"])
print(optimizer_output)

# Instantiate the problem and the model group
prob = om.Problem()

# Define flight conditions
Mach_number = 0.5 # You can change this if the user specifies a different Mach number
rho = 1.225
v = Mach_number * 340  # freestream speed, m/s
Re_c = rho * v / 1.81e-5  # Reynolds number / characteristic length, 1/m

indep_var_comp = om.IndepVarComp()
indep_var_comp.add_output("v", val=v, units="m/s")  # Freestream Velocity
indep_var_comp.add_output(
    "alpha", val=0.0, units="deg"
) 
indep_var_comp.add_output("Mach_number", val=Mach_number)  # Freestream Mach number
indep_var_comp.add_output("re", val=Re_c, units="1/m")  # Freestream Reynolds number times chord length
indep_var_comp.add_output("rho", val=rho, units="kg/m**3")  # Freestream air density
indep_var_comp.add_output("cg", val=np.zeros((3)), units="m")  # Aircraft center of gravity
prob.model.add_subsystem("flight_vars", indep_var_comp, promotes=["*"])

# Setup OpenAeroStruct model
name = surface["name"]

# Add geometry group to

In [12]:
#Write the chunks of code into the template file and run it
template_file = "RunOAS_template.py"

#It should then find """Part 1: PUT THE BASELINE MESH OF THE WING HERE""" then add print("hi") after it
with open(template_file, "r") as file:
    template_code = file.read()
    
    # Replace all placeholders with the generated code
    template_code = template_code.replace(
        '"""Part 1: PUT THE BASELINE MESH OF THE WING HERE"""',
        f'"""Part 1: PUT THE BASELINE MESH OF THE WING HERE"""\n{mesher_output["python_code"]}'
    )
    template_code = template_code.replace(
        '"""Part 2:  DO THE GEOMETRY SETUP HERE"""',
        f'"""Part 2:  DO THE GEOMETRY SETUP HERE"""\n{geometry_output["python_code"]}'
    )
    template_code = template_code.replace(
        '"""Part 3: PUT THE OPTIMIZER HERE """',
        f'"""Part 3: PUT THE OPTIMIZER HERE """\n{optimizer_output["python_code"]}'
    )

# Write the modified code to a new file instead of modifying the template
run_oas_file = "RunOAS.py"
with open(run_oas_file, "w") as file:
    file.write(template_code)

print("Created RunOAS.py file with all generated code sections")

Created RunOAS.py file with all generated code sections


In [13]:
# Now run the script using a subprocess, also capture all the outputs, and save it as a text file.
output_file = "output.txt"
with open(output_file, "w") as file:
    # Run the script and capture the output
    process = subprocess.Popen(["python3", run_oas_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # Decode the output and write it to the file
    file.write(stdout.decode())
    file.write(stderr.decode())

### After Running the Optimization, the LLM should then have access to the plots then analyze the results

In [14]:
HTML_Report = "RunOAS_out/reports/opt_report.html"
pdf_output_path = "./Figures/Opt_History.pdf"

# Convert HTML report to PDF using pandoc
try:
    # Check if the HTML file exists
    if not os.path.exists(HTML_Report):
        print(f"Error: HTML report not found at {HTML_Report}")
    else:
        print("Converting HTML report to PDF...")
        # Use subprocess to call pandoc for the conversion
        result = subprocess.run(
            [
                "pandoc", HTML_Report,
                "-o", pdf_output_path,
                "--pdf-engine=/Library/TeX/texbin/pdflatex",
                "-V", "geometry:landscape,a4paper",  # Landscape + specific paper size
                "-V", "geometry:margin=5mm",        # Reduce margins
                "-V", "geometry:includeheadfoot",    # Use full page area
                "--variable", "mainfont=Helvetica",  # More compact font
                "--variable", "fontsize=5pt"        # Smaller base font size
            ],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print(f"Successfully converted to PDF: {pdf_output_path}")
        else:
            print(f"Error converting to PDF: {result.stderr}")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Converting HTML report to PDF...
Successfully converted to PDF: ./Figures/Opt_History.pdf


In [15]:
def run_plot_wing(file_path):
    """
    Execute the plot_wing command on a specified aero.db file
    
    Args:
        file_path (str): Path to the aero.db file
    """
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist.")
        return
        
    try:
        # Run plot_wing command and capture output
        result = subprocess.run(["plot_wing", file_path], 
                               capture_output=True, 
                               text=True, 
                               check=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")
    except FileNotFoundError:
        print("Error: 'plot_wing' command not found. Please ensure it's installed and in your PATH and follow the changes to the plot_wing function shown in readme.")

# Example usage
file_path = "RunOAS_out/aero.db"
run_plot_wing(file_path)

In [17]:
retriever_setup = RetrievalAgent()
retriever_output = retriever_setup.execute_task(f"""Here is the rewritten optimization problem: {reformulator_output}""")
print(retriever_output["Retrieved_Information"])

# Run the RAG retriever
RAG_chunks = run_retriever(retriever_output["Retrieved_Information"])

# Pass all the retrived chunks page_content into a single string
retrieved_chunks = "\n".join([chunk.page_content for chunk in RAG_chunks])
print(retrieved_chunks)

Please provide detailed information regarding the following variables used in aerodynamic wing optimization: Taper ratio, Twist angle, Sweep angle, Wing Area (S), and Wing Span (b).  Specifically, I need definitions, typical ranges, units of measurement, and their impact on aerodynamic performance and optimization processes.
The latter approach is only viable when the designer has enough experience to know what consti-
tutes a desirable lift distribution. The ﬁrst approach is often combined with numerical optimization.
Wing lift distribution is an important consideration in wing design. The spanwise lift distribu-
tion is closely related to the wing geometry and determines wing performance characteristics such
as induced drag, structural weight, and stalling characteristics. A reasonable lift andCl distribution
provides a good starting point in the design of a wing.
6.1 Wing Design Parameters
The major wing design parameters can be divided into two levels. The ﬁrst level design paramet

In [18]:
# Optimization Setup
error_gen_flag = False
while not error_gen_flag:
    ResultsPrompt  = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}, and the optimization results are as follows: {optimizer_output}, here is some retrieved information: {RAG_chunks}"""
    results_setup = ResultsReaderAgent()
    results_output = results_setup.execute_task(ResultsPrompt)

    if results_output is not None:
        error_gen_flag = True


In [19]:
results_output

{'Analysis': 'The optimization run failed to meet the CL=2.0 constraint, as the final CL value was 0.796, significantly lower than the target. The optimization aimed to minimize CD, but the final CD value is quite high at 11.836. The lift distribution plot shows that it is not elliptical, which is expected for drag minimization. Additionally, the twist distribution seems constant, which may not be optimal for achieving the target CL while minimizing drag. The taper ratio is at the lower bound, and the sweep is at 14.5 degrees.',
 'Recommendations': '1.  **Increase Max Iterations:** Increase `maxiter` because the solver is having trouble converging.\n2.  **Constraint Scaling:** Ensure that the constraint is properly scaled to avoid numerical issues. A small change in the constraint value may lead to a large change in the objective function and may cause the solver to fail.\n3.  **Revise Bounds:** Review and revise the bounds for design variables. For instance, the lower bound for taper 

In [21]:
error_gen_flag = False
while not error_gen_flag:
    ReportPrompt  = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}, the analysis by the LLM is {results_output}"""

    report_setup = ReportWriter()
    report_output = report_setup.execute_task(ReportPrompt)

    if report_output is not None and report_output.get("ReportText") is not None:
        error_gen_flag = True

In [22]:
#Write the latex output into /Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/Figures/Report.tex and replace the original file.

report_file = "Figures/Report.tex"
with open(report_file, "w") as file:
    file.write(report_output["ReportText"])